In [52]:
import spacy 
from spacy.pipeline import EntityRuler
from spacy import displacy
import pandas as pd
import numpy as np 


In [53]:
df = pd.read_csv("../data/sample_resumes/Resume/resumeDataSet2_transformed_filtered.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()


,ID,Resume_str,Resume_html,Category
184,628,Skills VISA B1-VISA (USA) Onsite Visits to Swe...,NaN,DevOps Engineer
211,710,TECHNICAL EXPERTISE â¢ DB Languages: SQL â¢ ...,NaN,Database
320,861,"SOFTWARE SKILLS: Languages: C, C++ & java Oper...",NaN,Blockchain
2,322,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",NaN,Java Developer
198,642,CORE COMPETENCIES ~ Ant ~ Maven ~ GIT ~ Bitbuc...,NaN,DevOps Engineer


In [54]:
# import en_core_web_lg


# nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("ner")

ruler = nlp.add_pipe("entity_ruler", before='ner')

# ner = nlp.add_pipe("ner")

#Skills for entities 

SweSkills = ["Python", "Java", "JavaScript", "C++", "React", "Angular", "Node.js", "Git", "CI/CD"]
DsSkills = ["Python", "R", "SQL", "Machine Learning", "Data Analysis", "Statistics", "TensorFlow", "PyTorch"]
DoSkills = ["Linux", "Docker", "Kubernetes", "AWS", "Azure", "CI/CD", "Jenkins", "Terraform", "Ansible"]

patterns = []   

for skill in SweSkills:
    patterns.append({"label": "SWE", "pattern": skill})
for skill in DsSkills:
    patterns.append({"label": "DATASCIENCE", "pattern": skill})
for skill in DoSkills:
    patterns.append({"label": "DEVOPS", "pattern": skill})

ruler.add_patterns(patterns)

doc = nlp("Hello I love me some Angular some python, some other shit too like Terraform, DEVOPS, and idk some other shit and data sciennce")

# resume_text = data['']

for ent in doc.ents:
    print(ent.text, ent.label_)

# print(data['Resume_str'].iloc[0])

Angular SWE
Terraform DEVOPS
DEVOPS ORG


In [55]:
# For the training data
from spacy.training import Example

# Testing this out rq
def training_data(data):
    train_data = []
    for text in data['Resume_str']:
        doc = nlp(text)
        entities = []

        for ent in doc.ents:
            entities.append([ent.start_char, ent.end_char, ent.label_])
        train_data.append([text, {"entities": entities }])

    return train_data

In [56]:
# Used to better visualize the text etracted and seeing the enteties. 
# from pprint import pprint
train_data = training_data(data)
for i, (text, annot) in enumerate(train_data):  
    print(f"\nExample {i+1}:")
    print("Text preview:", text, "...\n")  

    for start, end, label in annot["entities"]:
        span_text = text[start:end]
        print(f"  - '{span_text}' ({label}) [{start}:{end}]")
# pprint(train_data) 


Example 1:
Text preview: Skills VISA B1-VISA (USA) Onsite Visits to Sweden & US (Seattle) Education Details 
January 2013 Post Graduate Diploma Information Technology Pune, Maharashtra Symbiosis Institute
January 2007 Bachelor of Engineering Electronics and Telecommunications Pune, Maharashtra Pune University
Cloud Operations Architect (DevOps) 

Cloud Operations Architect (DevOps) - DevOps
Skill Details 
Cloud Computing- Exprience - 48 months
Shell Scripting- Exprience - 96 months
Python- Exprience - 6 months
Automation- Exprience - 72 months
Solution Architect- Exprience - Less than 1 year months
Azure- Exprience - Less than 1 year months
AWS- Exprience - Less than 1 year monthsCompany Details 
company - DevOps
description - Type: DevOps Engineer.
Platform: AWS Cloud, Azure Cloud.
Services: AWS EC2, RDS, CloudFormation Template, Lambda, Dynamo DB,		Cloud Watch, Auto-scaling, Elastic Bean stalk, Appdynamics.

Here I manage Tibco Spotfire enterprise & Cloud Product support. Being the 

In [ ]:
# For the training examples:
from spacy.training import Example

def training_examples(training_data):
    examples = []
    for text, annotations in training_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)

examples = training_examples(train_data)
print(examples)

In [58]:
import random
import spacy
from spacy.training import Example

optimizer = nlp.initialize(get_examples=lambda: examples)

for i in range(10):  
    random.shuffle(examples)
    losses = {}
    for batch in spacy.util.minibatch(examples, size=8):
        nlp.update(batch, sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i}: {losses}")

ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.